In [1]:
import numpy as np
import sklearn
from sklearn import linear_model
import polars
from sklearn.model_selection import train_test_split
from scipy import sparse

In [24]:

df_it = polars.read_csv_batched("train.csv", has_header=True)
ys = []
Xs = []
idxs = []
for i in range(12):
    df = df_it.next_batches(1)[0]
    df[0, 0:10]
    print(df.shape)
    y = df[:, 1].to_numpy()
    ys.append(y)
    X = sparse.csr_matrix(np.float32(df[:, 2:].to_numpy()))
    X = sklearn.preprocessing.normalize(X)
    Xs.append(X)
    idx = df[:, 0]
    idxs.append(idx)

X = sparse.vstack(Xs)
y = np.hstack(ys)

del Xs
del ys

print(X.shape)
print(y.shape)
(X_train, X_test, y_train, y_test) = train_test_split(X, y, shuffle=False)
del X
del y
n_train = X_train.shape[0]
print(X_train.shape, X_test.shape)


# print(df.shape)
# print(df[:, 2:].mean())


(49998, 4098)
(49998, 4098)
(49998, 4098)
(49998, 4098)
(49998, 4098)
(49998, 4098)
(49998, 4098)
(49998, 4098)
(49998, 4098)
(49998, 4098)
(49998, 4098)
(49998, 4098)
(599976, 4096)
(599976,)
(449982, 4096) (149994, 4096)


In [26]:
from collections import Counter
counter = Counter()

c = 0
for line in open("dataset/archive/sentences.prepared.csv"):
    (rid, lang, sentence) = line.strip().split("\t", 2)
    if lang not in {"fra","eng", "ita", "deu", "esp", "por"}:
        continue
    c += 1
    if c > 100_000:
        break
    for chr in sentence:
        if ord(chr) > 128:
            counter[chr] += 1
letters = sorted(ord(letter) for (letter, count) in counter.most_common(100) if count >= 10)
print(", ".join(map(str, letters)))
#print(letters)

160, 171, 173, 187, 192, 196, 199, 200, 201, 202, 205, 214, 220, 223, 224, 225, 226, 227, 228, 231, 232, 233, 234, 235, 236, 237, 238, 239, 242, 243, 244, 245, 246, 249, 250, 251, 252, 333, 339, 8201, 8211, 8212, 8217, 8220, 8221, 8222, 8239


In [27]:
for C in [64]:
    model = sklearn.linear_model.LogisticRegression(max_iter=500, penalty='l2', multi_class='multinomial', C=C, verbose=1, class_weight='balanced') #, l1_ratio=0.1,) # penalty='elasticnet', solver='saga') 
    model.fit(X_train, y_train)
    print("\n\n\n\n\n----------")
    print(C)
    print((model.predict(X_train) == y_train).mean())
    print((model.predict(X_test) == y_test).mean())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        65552     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.24762D+06    |proj g|=  2.45139D+04

At iterate   50    f=  1.21037D+04    |proj g|=  3.55325D+02

At iterate  100    f=  4.94206D+03    |proj g|=  1.13277D+02

At iterate  150    f=  3.82850D+03    |proj g|=  1.43956D+01

At iterate  200    f=  3.43115D+03    |proj g|=  1.99731D+01

At iterate  250    f=  3.29184D+03    |proj g|=  1.39921D+01

At iterate  300    f=  3.23884D+03    |proj g|=  4.21846D+00

At iterate  350    f=  3.21968D+03    |proj g|=  1.86934D+00

At iterate  400    f=  3.21289D+03    |proj g|=  7.79969D-01

At iterate  450    f=  3.21017D+03    |proj g|=  1.19910D+00

At iterate  500    f=  3.20925D+03    |proj g|=  7.50076D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

/home/fulmicoton/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.0min finished


0.9997377672884693
0.9961465125271678


In [21]:
import time
time.sleep(100)
print("start")from sklearn import metrics
print(model.classes_)
print(sklearn.metrics.confusion_matrix(y_test, model.predict(X_test), labels=['deu', 'eng', 'fra',  'ita','nld', 'por', 'rus', 'spa']))

sklearn.metrics.confusion_matrix(y_test, model.predict(X_test), labels=['kor', 'jpn', 'cmn'])

['ara' 'cmn' 'deu' 'eng' 'fra' 'hin' 'ita' 'jpn' 'kor' 'nld' 'por' 'rus'
 'spa' 'swe' 'tur' 'vie']
[[13558    13     4     2    26     1     0     3]
 [   20 36656    23    10    27    10     0    20]
 [    2     9 11541     9     2     5     0    11]
 [    1    10    25 18634     4    28     0    41]
 [   14     9     2     4  3663     1     0     0]
 [    3     3    16    28     0  9036     0   104]
 [    0     0     0     0     0     0 21220     0]
 [    1     4    10    29     1    68     0  8532]]


array([[ 209,    0,    0],
       [   0, 5127,    3],
       [   0,    1, 1704]])

In [30]:

from sklearn import metrics
print(model.classes_)
print(sklearn.metrics.confusion_matrix(y_test, model.predict(X_test), labels=['deu', 'eng', 'fra',  'ita','nld', 'por', 'rus', 'spa']))

sklearn.metrics.confusion_matrix(y_test, model.predict(X_test), labels=['kor', 'jpn', 'cmn'])

['ara' 'cmn' 'deu' 'eng' 'fra' 'hin' 'ita' 'jpn' 'kor' 'nld' 'por' 'rus'
 'spa' 'swe' 'tur' 'vie']
[[13565     7     2     2    22     3     0     3]
 [   13 36698     8    12    20     7     0     9]
 [    1     6 11555     7     1     4     0     5]
 [    3     6    20 18666     4    22     0    29]
 [   11    12     2     4  3665     5     0     1]
 [    4     3     6    22     1  9064     0    89]
 [    0     0     0     0     0     0 21220     0]
 [    2     3     9    34     0    58     0  8539]]


array([[ 209,    0,    0],
       [   0, 5128,    3],
       [   0,    2, 1703]])

In [26]:
from sklearn import metrics
print(model.classes_)
print(sklearn.metrics.confusion_matrix(y_test, model.predict(X_test), labels=['deu', 'eng', 'fra',  'ita','nld', 'por', 'rus', 'spa']))

sklearn.metrics.confusion_matrix(y_test, model.predict(X_test), labels=['kor', 'jpn', 'cmn'])

['ara' 'cmn' 'deu' 'eng' 'fra' 'hin' 'ita' 'jpn' 'kor' 'nld' 'por' 'rus'
 'spa' 'swe' 'tur' 'vie']
[[13562    11     2     3    21     2     0     3]
 [   14 36684    11    13    21     5     0    15]
 [    1     6 11551     8     2     4     0     7]
 [    3     6    21 18662     5    22     0    30]
 [   11    12     2     4  3666     5     0     0]
 [    4     3     8    23     1  9060     0    90]
 [    0     0     0     0     0     0 21220     0]
 [    2     3     8    35     0    57     0  8539]]


array([[ 209,    0,    0],
       [   0, 5128,    3],
       [   0,    1, 1704]])

In [16]:
from sklearn import metrics
print(model.classes_)
print(sklearn.metrics.confusion_matrix(y_test, model.predict(X_test), labels=['deu', 'eng', 'fra',  'ita','nld', 'por', 'rus', 'spa']))

sklearn.metrics.confusion_matrix(y_test, model.predict(X_test), labels=['kor', 'jpn', 'cmn'])

['ara' 'cmn' 'deu' 'eng' 'fra' 'hin' 'ita' 'jpn' 'kor' 'nld' 'por' 'rus'
 'spa' 'swe' 'tur' 'vie']
[[ 6866     8     2     6    10     2     0     7]
 [    4 18245    22    11    30     3     0    22]
 [    0     2  5838     9     3     7     0    12]
 [    1     6    14  9446     0    27     0    40]
 [   10     7     1     3  1770     0     0     4]
 [    1     2    10    31     4  4517     0    58]
 [    0     0     0     0     0     0 10485     0]
 [    2     4    10    34     1    43     0  4133]]


array([[ 101,    0,    0],
       [   0, 2561,    5],
       [   0,    0,  812]])

In [27]:
y_predict = model.predict(X_test)
print(np.where((y_predict == y_test) == False)[0].shape)
print(y_predict.shape)
i = 0 
for row in list(np.where((y_predict == y_test) == False))[0]:
    i += 1
    print(y_test[row], y_predict[row], idx[int(n_train + row)])
    if i == 10:
        break

(565,)
(75000,)
ita por 8484775
ita spa 3553712
ita por 2614727
ita spa 3213159
tur nld 4129221
por spa 6992976
por spa 7888818
deu fra 843713
eng swe 1164430
por spa 972364


In [31]:
(LANG, DIM) = model.coef_.shape
print(model.coef_.shape)
coef = np.float32(model.coef_)

print(coef[0,0])
print(model.coef_[:,0])

f = open("src/weights.rs", "w")

f.write("#[derive(Clone, Copy, Debug, Eq, PartialEq)]\n")
f.write("pub enum Lang {\n")
for lang in model.classes_:
    f.write("\t%s,\n" % lang.capitalize(),)
f.write("}\n\n")

f.write("""
impl Lang {
    pub fn three_letter_code(self)-> &'static str {
        match self {
""")
for lang in model.classes_:
    f.write("\t\t\tLang::%s => \"%s\",\n" % (lang.capitalize(), lang))
f.write("\t\t}\t}\n}\n\n\n")


f.write("pub const LANGUAGES: [Lang; %d] = [\n\t" % LANG)
for lang in model.classes_:
    f.write("Lang::%s, " % lang.capitalize())
f.write("];\n\n")

f.write("pub const WEIGHTS: [f32; %d] = [\n" % (LANG * DIM))
for i in range(DIM):
    f.write("\t")
    for val in coef[:, i]:
        f.write("%f, " % val)
    f.write("\n")
f.write("];\n\n")


f.write("pub const INTERCEPTS: [f32; %d] = [\n\t" % LANG)
for val in model.intercept_:
    f.write("%f, " % val)
f.write("];\n\n")


f.flush()
f.close()


(16, 4096)
-0.0047384733
[-4.73847311e-03 -2.28266937e-03 -5.36394365e-01  5.88218350e-01
 -1.03133002e-01 -5.35878639e-04 -2.13580000e+00 -2.86555783e-03
 -1.19892844e-03 -3.64568806e-01 -3.39155160e-01 -5.44463552e-04
  8.38150022e-01  1.53047103e+00  5.64755284e-01 -3.03773764e-02]
